# Kapitel 2.5: Vererbung – Mehrfachvererbung & MRO
---

## **1. Mehrfachvererbung: Fähigkeiten kombinieren**

### **Was ist Mehrfachvererbung und warum braucht man sie?**

Bisher kannten wir nur die einfache Vererbung: Eine Kindklasse erbt von **genau einer** Elternklasse. Die Mehrfachvererbung erlaubt es einer Klasse, von **zwei oder mehr** Elternklassen gleichzeitig zu erben.

* **Syntax:** Die Elternklassen werden einfach durch Komma getrennt in den Klammern aufgelistet: `class Kind(Eltern1, Eltern2, Eltern3):`
* **Zweck:** Der Hauptzweck ist nicht, komplexe "ist ein"-Hierarchien zu bauen, sondern eher, verschiedene, voneinander unabhängige **Fähigkeiten oder Verhaltensweisen** in einer Klasse zu vereinen. Man spricht hier auch vom **"Mixin"-Muster**: Wir mischen die Fähigkeiten verschiedener Klassen zusammen.
* **Analogie:** Ein **Schweizer Taschenmesser**. Es ist eine Kombination aus unabhängigen Werkzeugen. Es erbt die Fähigkeit "schneiden" von einem `Messer`, "schrauben" von einem `Schraubendreher` und "öffnen" von einem `Dosenöffner`.
* **Analogie 2 (Zusammengesetztes Objekt):** Ein **Amphibienfahrzeug**. Es *ist ein* Landfahrzeug (es hat Räder und kann fahren) UND es *ist ein* Wasserfahrzeug (es hat einen Propeller und kann schwimmen). Es erbt die Fähigkeiten von beiden.

---

### **Kombination von Bewegungsarten**

Ein sehr intuitives Beispiel ist die Kombination von zwei fundamental unterschiedlichen Bewegungsarten.

In [3]:
class Fliegend:
    def fliegen(self):
        print("Bewegt sich durch die Luft.")

class Schwimmend:
    def schwimmen(self):
        print("Bewegt sich durch das Wasser.")

# Die Ente kann beides, also erbt sie von beiden Eltern
class Ente(Fliegend, Schwimmend):
    def quaken(self):
        print("Quak!")

# Testen wir die kombinierten Fähigkeiten
donald = Ente()
donald.fliegen()
donald.schwimmen()
donald.quaken()

Bewegt sich durch die Luft.
Bewegt sich durch das Wasser.
Quak!



---

### **Das "Mixin"-Muster für Funktionalität**

In der Softwareentwicklung nutzen wir oft kleine Klassen (Mixins), die nur eine einzige, wiederverwendbare Fähigkeit bereitstellen.

In [4]:
# Ein Mixin, das die Fähigkeit zum Protokollieren bereitstellt
class Protokollierbar:
    def log(self, nachricht):
        # In einer echten Anwendung würde hier in eine Datei geschrieben werden
        print(f"[LOG] {nachricht}")

# Ein zweites, unabhängiges Mixin für Authentifizierung
class Authentifizierbar:
    def authentifiziere(self, passwort):
        if passwort == "geheim123":
            # Beachten Sie: Eine Methode aus einem Mixin kann nicht wissen,
            # ob ein anderes Mixin ebenfalls vorhanden ist.
            # Ein direkter Aufruf wie self.log() wäre hier riskant.
            print("Authentifizierung erfolgreich.")
            return True
        else:
            print("Authentifizierung fehlgeschlagen.")
            return False

# Unsere Hauptklasse erbt nun beide Fähigkeiten
class WebRequest(Protokollierbar, Authentifizierbar):
    def daten_senden(self, daten, passwort):
        self.log("Datenversand wird versucht...") # geerbt von Protokollierbar
        if self.authentifiziere(passwort):        # geerbt von Authentifizierbar
            self.log(f"Daten '{daten}' werden gesendet.")
        else:
            self.log("Datenversand abgebrochen.")
            
# Instanz erstellen und testen
request = WebRequest()
request.daten_senden("Top Secret Info", "geheim123")

[LOG] Datenversand wird versucht...
Authentifizierung erfolgreich.
[LOG] Daten 'Top Secret Info' werden gesendet.



---
**Markdown-Zelle**
````markdown
### **`__init__` in mehreren Elternklassen**

Was passiert, wenn mehrere Elternklassen einen `__init__`-Konstruktor haben? `super()` allein kann hier verwirrend sein, da es einem klaren Pfad (der MRO) folgt und nicht unbedingt alle Konstruktoren aufruft. In solchen Fällen muss man die Konstruktoren der Eltern oft explizit über den Klassennamen aufrufen.



In [5]:
class Student:
    def __init__(self, matrikelnr):
        print("INIT Student...")
        self.matrikelnr = matrikelnr

class Angestellter:
    def __init__(self, personalnr):
        print("INIT Angestellter...")
        self.personalnr = personalnr
        
# Ein Werkstudent ist beides: Student UND Angestellter
class Werkstudent(Student, Angestellter):
    def __init__(self, matrikelnr, personalnr, studiengang):
        # Wir rufen die Konstruktoren beider Eltern explizit auf
        Student.__init__(self, matrikelnr)
        Angestellter.__init__(self, personalnr)
        
        # Eigenes Attribut hinzufügen
        self.studiengang = studiengang

# Test
ws = Werkstudent("12345", "A987", "Informatik")
print(f"Matrikelnr: {ws.matrikelnr}, Personalnr: {ws.personalnr}, Studiengang: {ws.studiengang}")

INIT Student...
INIT Angestellter...
Matrikelnr: 12345, Personalnr: A987, Studiengang: Informatik


In [6]:
# Übung 1

### **Transfer auf das Projekt: Das multifunktionale `Raumschiff`**

Nachdem wir das Konzept an verschiedenen Beispielen verstanden haben, wenden wir es nun auf unser `Raumschiff`-Projekt an. Wir nutzen das "Mixin"-Muster, um einem Schiff optionale Fähigkeiten zu verleihen.

In [7]:
# Unsere bekannte Basisklasse
class Schiffsklasse:
    def __init__(self, name):
        self.name = name
    def status_report(self):
        print(f"Status für {self.name}: Alle Systeme nominal.")

# Mixin 1: Scan-Fähigkeit
class MineralienScanner:
    def scanne_planet(self, planet):
        print(f"Scanner wird aktiviert. Scanne Planet {planet} auf wertvolle Mineralien...")

# Mixin 2: Tarn-Fähigkeit
class Tarnvorrichtung:
    def aktiviere_tarnung(self):
        print(f"Tarnfeld wird um {self.name} aufgebaut. Schiff ist nun unsichtbar.")
        
# Ein Aufklärungsschiff erbt von der Basisklasse UND den beiden Mixins
class AufklaerungsSchiff(Schiffsklasse, MineralienScanner, Tarnvorrichtung):
    def __init__(self, name):
        # Wir müssen nur den Konstruktor der Haupt-Basisklasse aufrufen
        super().__init__(name)

# Testen
raven = AufklaerungsSchiff("USS Raven")

# Fähigkeit von Schiffsklasse
raven.status_report()

# Fähigkeit von MineralienScanner
raven.scanne_planet("Vulcan")

# Fähigkeit von Tarnvorrichtung
raven.aktiviere_tarnung()


Status für USS Raven: Alle Systeme nominal.
Scanner wird aktiviert. Scanne Planet Vulcan auf wertvolle Mineralien...
Tarnfeld wird um USS Raven aufgebaut. Schiff ist nun unsichtbar.


---
---

## **2. Method Resolution Order (MRO): Pythons Regelwerk für Vererbung**

### **Das Problem des Konflikts**

Mehrfachvererbung ist mächtig, aber sie schafft ein potenzielles Problem: **Was passiert, wenn zwei oder mehr Elternklassen eine Methode mit exakt demselben Namen haben?**

**Analogie:** Ihr Vater sagt: "Räum dein Zimmer so auf!", Ihre Mutter sagt: "Nein, räum es so auf!". Auf wen hören Sie? Sie brauchen eine klare Regel.

Genau diese Regel ist in Python die **Method Resolution Order (MRO)**.

* **Definition:** Die MRO ist eine **eindeutige, lineare Reihenfolge** aller Klassen in einer Vererbungshierarchie. Sie legt fest, in welcher Reihenfolge Python nach einer aufgerufenen Methode sucht.
* **Funktionsweise:** Wenn Sie eine Methode aufrufen (z.B. `obj.methode()`), durchsucht Python die Klassen in der Reihenfolge der MRO. Sobald es die Methode in einer Klasse findet, wird diese ausgeführt und die Suche **sofort gestoppt**.
* **Algorithmus:** Python verwendet den **C3-Linearisierungsalgorithmus**. Die vereinfachte Regel lautet: Python geht die Elternklassen in der Definition **von links nach rechts** durch und durchsucht dabei zuerst die komplette Hierarchie eines Elternteils, bevor es zum nächsten übergeht, ohne dabei eine Klasse doppelt zu besuchen.

---

### **Die MRO untersuchen**

Python gibt uns Werkzeuge, um diese Reihenfolge genau zu analysieren:

* **`Klassenname.mro()`:** Gibt die MRO als Liste von Klassen zurück.
* **`help(Klassenname)`:** Zeigt eine ausführliche Hilfe an, die auch einen Abschnitt "Method resolution order" enthält.

---

### **Ein einfacher Namenskonflikt**

In [8]:
class Vater:
    def kochen(self):
        print("Der Vater kocht Gulasch nach seinem Rezept.")

class Mutter:
    def kochen(self):
        print("Die Mutter kocht eine exzellente Gemüselasagne.")

# Das Kind erbt zuerst von Vater, dann von Mutter
class Kind(Vater, Mutter):
    pass

# Test
kind = Kind()
kind.kochen()

# Analyse
print("\nDie MRO für die Klasse 'Kind' ist:")
# Die Reihenfolge ist: Kind -> Vater -> Mutter -> object
print(Kind.mro())

Der Vater kocht Gulasch nach seinem Rezept.

Die MRO für die Klasse 'Kind' ist:
[<class '__main__.Kind'>, <class '__main__.Vater'>, <class '__main__.Mutter'>, <class 'object'>]



**Analyse:** Da `Vater` in der Definition `class Kind(Vater, Mutter):` an erster Stelle steht, wird seine `kochen()`-Methode als erste gefunden und ausgeführt. Die Suche stoppt, bevor Python überhaupt bei `Mutter` ankommt.



---

### **Eine tiefere Hierarchie**
Die MRO geht immer erst in die Tiefe, bevor sie zur Seite geht.

In [9]:
class A: pass
class B(A): pass
class C(B): pass # C erbt von B, B erbt von A

class D: pass

# E erbt zuerst von der tiefen Hierarchie (C), dann von D
class E(C, D): pass

print("MRO für Klasse E:")
# Python durchläuft zuerst die gesamte Kette von C (C -> B -> A), bevor es D prüft.
print(E.mro())

MRO für Klasse E:
[<class '__main__.E'>, <class '__main__.C'>, <class '__main__.B'>, <class '__main__.A'>, <class '__main__.D'>, <class 'object'>]


In [10]:
# Übung


---

### **Transfer auf das Projekt: Konflikt im `Raumschiff`**

Was passiert, wenn sowohl unser `MineralienScanner` als auch die `Tarnvorrichtung` eine Methode namens `system_check()` haben?

In [11]:
class Schiffsklasse:
    def __init__(self, name):
        self.name = name

class MineralienScanner:
    def system_check(self):
        print("[Scanner-Check] Sensoren kalibriert und online.")

class Tarnvorrichtung:
    def system_check(self):
        print("[Tarn-Check] Emitterspulen sind einsatzbereit.")
        
# Wir erben zuerst von Schiffsklasse, dann Scanner, dann Tarnvorrichtung
class AufklaerungsSchiff(Schiffsklasse, MineralienScanner, Tarnvorrichtung):
    pass

# Test
raven = AufklaerungsSchiff("Raven")
raven.system_check() # Welche Methode wird ausgeführt?

print("\nMRO für AufklaerungsSchiff:")
# Der Pfad ist: AufklaerungsSchiff -> Schiffsklasse -> MineralienScanner -> ...
print(AufklaerungsSchiff.mro())

[Scanner-Check] Sensoren kalibriert und online.

MRO für AufklaerungsSchiff:
[<class '__main__.AufklaerungsSchiff'>, <class '__main__.Schiffsklasse'>, <class '__main__.MineralienScanner'>, <class '__main__.Tarnvorrichtung'>, <class 'object'>]



**Analyse:** Die MRO durchsucht `AufklaerungsSchiff` (leer), dann `Schiffsklasse` (hat kein `system_check`), dann `MineralienScanner`. Dort wird sie fündig und führt dessen Methode aus. `Tarnvorrichtung` wird nie erreicht.

---
---

## **3. Das "Diamond Problem": Der Härtetest für die MRO**

### **Was ist das Diamond Problem?**

Das "Diamond Problem" ist der klassische Härtefall der Mehrfachvererbung. Es tritt auf, wenn eine Klasse von zwei Klassen erbt, die ihrerseits beide von einer gemeinsamen Basisklasse erben.

Die Vererbungshierarchie sieht aus wie ein Diamant (eine Raute):

```
      A  (Basisklasse, z.B. Tier)
     / \
    /   \
   B     C (z.B. Pflanzenfresser, Fleischfresser)
    \   /
     \ /
      D  (z.B. Allesfresser)
```
**Das Problem der Ambiguität:**
* Wenn `D` eine Methode von `A` aufruft, soll es den Weg über `B` oder über `C` gehen?
* Noch wichtiger: Wenn die Konstruktoren von `B` und `C` jeweils den Konstruktor von `A` mit `super().__init__()` aufrufen, wird dann der `__init__` von `A` am Ende zweimal ausgeführt?

Pythons C3-Linearisierungs-Algorithmus (MRO) ist exakt dafür gemacht, dieses Problem elegant zu lösen. Er stellt sicher, dass **jede Klasse in der Hierarchie nur genau einmal durchlaufen wird**.

---

### **Die Diamant-Struktur im Code**

In [12]:
class A:
    def __init__(self):
        print("Konstruktor von A aufgerufen.")

class B(A):
    def __init__(self):
        print("Konstruktor von B startet...")
        super().__init__()

class C(A):
    def __init__(self):
        print("Konstruktor von C startet...")
        super().__init__()

class D(B, C):
    def __init__(self):
        print("Konstruktor von D startet...")
        super().__init__()

print("Erstelle eine Instanz von D:")
instanz_d = D()

print("\nMRO für D:")
print([cls.__name__ for cls in D.mro()])

Erstelle eine Instanz von D:
Konstruktor von D startet...
Konstruktor von B startet...
Konstruktor von C startet...
Konstruktor von A aufgerufen.

MRO für D:
['D', 'B', 'C', 'A', 'object']



**Analyse:** Obwohl der `super()`-Pfad sowohl von `B` als auch von `C` zu `A` führt, wird `"Konstruktor von A aufgerufen."` nur **ein einziges Mal** ausgegeben. Die MRO `[D, B, C, A, object]` sorgt für einen klaren, linearen Pfad und verhindert doppelte Aufrufe.

In [13]:
# Übung


---

### **Transfer auf das Projekt: Der `Kampfschiff`-Diamant**

Wir konstruieren einen Diamanten mit unseren Schiffsklassen, um das Konzept zu festigen.

In [14]:
class Schiffsklasse:
    def __init__(self):
        print("INIT: Schiffsklasse (Basis)")

class Waffenplattform(Schiffsklasse):
    def __init__(self):
        print("INIT: Waffenplattform startet...")
        super().__init__()

class Antriebssystem(Schiffsklasse):
    def __init__(self):
        print("INIT: Antriebssystem startet...")
        super().__init__()

# Ein Kampfschiff erbt die Eigenschaften beider Subsysteme
class Kampfschiff(Waffenplattform, Antriebssystem):
    def __init__(self):
        print("INIT: Kampfschiff startet...")
        super().__init__()

print("Erstelle ein Kampfschiff:")
kriegsschiff = Kampfschiff()

print("\nMRO für Kampfschiff:")
print([cls.__name__ for cls in Kampfschiff.mro()])

Erstelle ein Kampfschiff:
INIT: Kampfschiff startet...
INIT: Waffenplattform startet...
INIT: Antriebssystem startet...
INIT: Schiffsklasse (Basis)

MRO für Kampfschiff:
['Kampfschiff', 'Waffenplattform', 'Antriebssystem', 'Schiffsklasse', 'object']



---
---

## **4. Tages-Challenge: Anwendung im Schüler-Projekt (`Smart Grid`)**

Jetzt ist es an der Zeit, alle heutigen Konzepte in Ihrem `Intelligentes Stromnetz`-Projekt zu kombinieren.

**Ihre umfassende Aufgabe:**

1.  **Erstellen Sie zwei "Mixin"-Klassen:**
    * `Speicherbar`: Mit einer Methode `strom_speichern(menge)`, die ausgibt, wie viel Strom gespeichert wird. Diese Klasse soll auch einen `__init__` haben, der das Attribut `speicher_kapazitaet` setzt und eine Nachricht ausgibt.
    * `Regelbar`: Mit einer Methode `leistung_anpassen(prozent)`, die ausgibt, auf wie viel Prozent die Leistung angepasst wird.

2.  **Bauen Sie eine Diamant-Struktur auf:**
    * Ihre Basisklasse `Energieerzeuger` bleibt bestehen.
    * Erstellen Sie eine Klasse `Gaskraftwerk`, die von `Energieerzeuger` erbt.
    * Erstellen Sie eine Klasse `Batteriespeicher`, die von `Energieerzeuger` UND dem Mixin `Speicherbar` erbt.

3.  **Kombinieren Sie alles in einer finalen Klasse:**
    * Erstellen Sie eine Klasse `HybridKraftwerk`, die von `Gaskraftwerk`, `Batteriespeicher` und dem Mixin `Regelbar` erbt.
    * Schreiben Sie einen `__init__`-Konstruktor für `HybridKraftwerk`, der alle notwendigen Attribute für alle seine Elternklassen entgegennimmt. Rufen Sie die Konstruktoren der Eltern so auf, dass alle Attribute korrekt initialisiert werden. (Tipp: `super()` funktioniert für die lineare Kette, aber für Mixins muss man eventuell den `__init__` direkt aufrufen: `Speicherbar.__init__(self, ...)`).

4.  **Analysieren und Testen:**
    * Erstellen Sie eine Instanz Ihres `HybridKraftwerks`.
    * Rufen Sie alle geerbten Methoden auf (`strom_speichern`, `leistung_anpassen` etc.).
    * Geben Sie die MRO der `HybridKraftwerk`-Klasse aus und erklären Sie in einem Kommentar, warum die Reihenfolge so ist, wie sie ist.
```
